# My Lab

In [1]:
import h5py
import numpy as np
from tqdm import tqdm

In [2]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEXMEGA'

In [3]:
%%bash
cd ../../../hardware/victims/firmware/
mkdir -p simpleserial-base-mylab && cp -r simpleserial-base/* $_
cd simpleserial-base-mylab

In [ ]:
%%bash -s "$PLATFORM"
cd ../../../hardware/victims/firmware/simpleserial-base-mylab
make PLATFORM=$1 CRYPTO_TARGET=NONE -j

In [ ]:
import chipwhisperer as cw
try:
    if not scope.connectStatus:
        scope.con()
except NameError:
    scope = cw.scope()

try:
    target = cw.target(scope)
except IOError:
    print("INFO: Caught exception on reconnecting to target - attempting to reconnect to scope first.")
    print("INFO: This is a work-around when USB has died without Python knowing. Ignore errors above this line.")
    scope = cw.scope()
    target = cw.target(scope)

print("INFO: Found ChipWhisperer😍")

if "STM" in PLATFORM or PLATFORM == "CWLITEARM" or PLATFORM == "CWNANO":
    prog = cw.programmers.STM32FProgrammer
elif PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
    prog = cw.programmers.XMEGAProgrammer
else:
    prog = None
    
import time
time.sleep(0.05)
scope.default_setup()
def reset_target(scope):
    if PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
        scope.io.pdic = 'low'
        time.sleep(0.05)
        scope.io.pdic = 'high_z' #XMEGA doesn't like pdic driven high
        time.sleep(0.05)
    else:  
        scope.io.nrst = 'low'
        time.sleep(0.05)
        scope.io.nrst = 'high_z'
        time.sleep(0.05)

In [ ]:
cw.program_target(scope, prog, "../../../hardware/victims/firmware/simpleserial-base-mylab/simpleserial-base-{}.hex".format(PLATFORM))

In [7]:
def sbox_word1(pt, key):
    #参数SBOX和FK
    sbox=bytearray(b'\xd6\x90\xe9\xfe\xcc\xe1\x3d\xb7\x16\xb6\x14\xc2\x28\xfb\x2c\x05\x2b\x67\x9a\x76\x2a\xbe\x04\xc3\xaa\x44\x13\x26\x49\x86\x06\x99\x9c\x42\x50\xf4\x91\xef\x98\x7a\x33\x54\x0b\x43\xed\xcf\xac\x62\xe4\xb3\x1c\xa9\xc9\x08\xe8\x95\x80\xdf\x94\xfa\x75\x8f\x3f\xa6\x47\x07\xa7\xfc\xf3\x73\x17\xba\x83\x59\x3c\x19\xe6\x85\x4f\xa8\x68\x6b\x81\xb2\x71\x64\xda\x8b\xf8\xeb\x0f\x4b\x70\x56\x9d\x35\x1e\x24\x0e\x5e\x63\x58\xd1\xa2\x25\x22\x7c\x3b\x01\x21\x78\x87\xd4\x00\x46\x57\x9f\xd3\x27\x52\x4c\x36\x02\xe7\xa0\xc4\xc8\x9e\xea\xbf\x8a\xd2\x40\xc7\x38\xb5\xa3\xf7\xf2\xce\xf9\x61\x15\xa1\xe0\xae\x5d\xa4\x9b\x34\x1a\x55\xad\x93\x32\x30\xf5\x8c\xb1\xe3\x1d\xf6\xe2\x2e\x82\x66\xca\x60\xc0\x29\x23\xab\x0d\x53\x4e\x6f\xd5\xdb\x37\x45\xde\xfd\x8e\x2f\x03\xff\x6a\x72\x6d\x6c\x5b\x51\x8d\x1b\xaf\x92\xbb\xdd\xbc\x7f\x11\xd9\x5c\x41\x1f\x10\x5a\xd8\x0a\xc1\x31\x88\xa5\xcd\x7b\xbd\x2d\x74\xd0\x12\xb8\xe5\xb4\xb0\x89\x69\x97\x4a\x0c\x96\x77\x7e\x65\xb9\xf1\x09\xc5\x6e\xc6\x84\x18\xf0\x7d\xec\x3a\xdc\x4d\x20\x79\xee\x5f\x3e\xd7\xcb\x39\x48')                   
    fk0=bytearray(b'\xa3\xb1\xba\xc6')
    fk1=bytearray(b'\x56\xaa\x33\x50')
    fk2=bytearray(b'\x67\x7d\x91\x97')
    fk3=bytearray(b'\xb2\x70\x22\xdc')
    #传入明文
    p1=bytearray([pt[4],pt[5],pt[6],pt[7]])
    p2=bytearray([pt[8],pt[9],pt[10],pt[11]])
    p3=bytearray([pt[12],pt[13],pt[14],pt[15]])
    #传入主密钥
    mk0=bytearray([key[0],key[1],key[2],key[3]])
    mk1=bytearray([key[4],key[5],key[6],key[7]])
    mk2=bytearray([key[8],key[9],key[10],key[11]])
    mk3=bytearray([key[12],key[13],key[14],key[15]])
    #主密钥和FK逐比特异或
    k0=bytearray([mk0[0]^fk0[0], mk0[1]^fk0[1], mk0[2]^fk0[2], mk0[3]^fk0[3]])
    k1=bytearray([mk1[0]^fk1[0], mk1[1]^fk1[1], mk1[2]^fk1[2], mk1[3]^fk1[3]])
    k2=bytearray([mk2[0]^fk2[0], mk2[1]^fk2[1], mk2[2]^fk2[2], mk2[3]^fk2[3]])
    k3=bytearray([mk3[0]^fk3[0], mk3[1]^fk3[1], mk3[2]^fk3[2], mk3[3]^fk3[3]])
    #参数CK
    ck0=bytearray(b'\x00\x07\x0e\x15')
    keyboxin=bytearray([k1[0]^k2[0]^k3[0]^ck0[0], k1[1]^k2[1]^k3[1]^ck0[1], k1[2]^k2[2]^k3[2]^ck0[2], k1[3]^k2[3]^k3[3]^ck0[3]])
    keyboxout=bytearray([sbox[keyboxin[0]], sbox[keyboxin[1]], sbox[keyboxin[2]], sbox[keyboxin[3]]])
    left7=bytearray([keyboxout[2]<<7&255, keyboxout[3]<<7&255, keyboxout[0]<<7&255, keyboxout[1]<<7&255])
    right1=bytearray([keyboxout[3]>>1, keyboxout[0]>>1, keyboxout[1]>>1, keyboxout[2]>>1])
    left5=bytearray([keyboxout[1]<<5&255, keyboxout[2]<<5&255, keyboxout[3]<<5&255, keyboxout[0]<<5&255])
    right3=bytearray([keyboxout[2]>>3, keyboxout[3]>>3, keyboxout[0]>>3, keyboxout[1]>>3])
    rotation23=bytearray([left7[0]^right1[0], left7[1]^right1[1], left7[2]^right1[2], left7[3]^right1[3]])                   
    rotation13=bytearray([left5[0]^right3[0], left5[1]^right3[1], left5[2]^right3[2], left5[3]^right3[3]])
    linear=bytearray([keyboxout[0]^rotation13[0]^rotation23[0], keyboxout[1]^rotation13[1]^rotation23[1], keyboxout[2]^rotation13[2]^rotation23[2], keyboxout[3]^rotation13[3]^rotation23[3]])
    rk0=bytearray([k0[0]^linear[0], k0[1]^linear[1], k0[2]^linear[2], k0[3]^linear[3]])
    p=bytearray([p1[0]^p2[0]^p3[0], p1[1]^p2[1]^p3[1], p1[2]^p2[2]^p3[2], p1[3]^p2[3]^p3[3]])
    pboxin=bytearray([p1[0]^p2[0]^p3[0]^rk0[0], p1[1]^p2[1]^p3[1]^rk0[1], p1[2]^p2[2]^p3[2]^rk0[2], p1[3]^p2[3]^p3[3]^rk0[3]])
    label=bytearray([sbox[pboxin[0]], sbox[pboxin[1]], sbox[pboxin[2]], sbox[pboxin[3]]])
    return p, rk0, label

In [7]:
def sbox_word2(pt, key):
    #参数SBOX和FK
    sbox=bytearray(b'\xd6\x90\xe9\xfe\xcc\xe1\x3d\xb7\x16\xb6\x14\xc2\x28\xfb\x2c\x05\x2b\x67\x9a\x76\x2a\xbe\x04\xc3\xaa\x44\x13\x26\x49\x86\x06\x99\x9c\x42\x50\xf4\x91\xef\x98\x7a\x33\x54\x0b\x43\xed\xcf\xac\x62\xe4\xb3\x1c\xa9\xc9\x08\xe8\x95\x80\xdf\x94\xfa\x75\x8f\x3f\xa6\x47\x07\xa7\xfc\xf3\x73\x17\xba\x83\x59\x3c\x19\xe6\x85\x4f\xa8\x68\x6b\x81\xb2\x71\x64\xda\x8b\xf8\xeb\x0f\x4b\x70\x56\x9d\x35\x1e\x24\x0e\x5e\x63\x58\xd1\xa2\x25\x22\x7c\x3b\x01\x21\x78\x87\xd4\x00\x46\x57\x9f\xd3\x27\x52\x4c\x36\x02\xe7\xa0\xc4\xc8\x9e\xea\xbf\x8a\xd2\x40\xc7\x38\xb5\xa3\xf7\xf2\xce\xf9\x61\x15\xa1\xe0\xae\x5d\xa4\x9b\x34\x1a\x55\xad\x93\x32\x30\xf5\x8c\xb1\xe3\x1d\xf6\xe2\x2e\x82\x66\xca\x60\xc0\x29\x23\xab\x0d\x53\x4e\x6f\xd5\xdb\x37\x45\xde\xfd\x8e\x2f\x03\xff\x6a\x72\x6d\x6c\x5b\x51\x8d\x1b\xaf\x92\xbb\xdd\xbc\x7f\x11\xd9\x5c\x41\x1f\x10\x5a\xd8\x0a\xc1\x31\x88\xa5\xcd\x7b\xbd\x2d\x74\xd0\x12\xb8\xe5\xb4\xb0\x89\x69\x97\x4a\x0c\x96\x77\x7e\x65\xb9\xf1\x09\xc5\x6e\xc6\x84\x18\xf0\x7d\xec\x3a\xdc\x4d\x20\x79\xee\x5f\x3e\xd7\xcb\x39\x48')                   
    fk0=bytearray(b'\xa3\xb1\xba\xc6')
    fk1=bytearray(b'\x56\xaa\x33\x50')
    fk2=bytearray(b'\x67\x7d\x91\x97')
    fk3=bytearray(b'\xb2\x70\x22\xdc')
    #传入明文
    p1=bytearray([pt[4],pt[5],pt[6],pt[7]])
    p2=bytearray([pt[8],pt[9],pt[10],pt[11]])
    p3=bytearray([pt[12],pt[13],pt[14],pt[15]])
    #传入主密钥
    mk0=bytearray([key[0],key[1],key[2],key[3]])
    mk1=bytearray([key[4],key[5],key[6],key[7]])
    mk2=bytearray([key[8],key[9],key[10],key[11]])
    mk3=bytearray([key[12],key[13],key[14],key[15]])
    #主密钥和FK逐比特异或
    k0=bytearray([mk0[0]^fk0[0], mk0[1]^fk0[1], mk0[2]^fk0[2], mk0[3]^fk0[3]])
    k1=bytearray([mk1[0]^fk1[0], mk1[1]^fk1[1], mk1[2]^fk1[2], mk1[3]^fk1[3]])
    k2=bytearray([mk2[0]^fk2[0], mk2[1]^fk2[1], mk2[2]^fk2[2], mk2[3]^fk2[3]])
    k3=bytearray([mk3[0]^fk3[0], mk3[1]^fk3[1], mk3[2]^fk3[2], mk3[3]^fk3[3]])
    #参数CK
    ck0=bytearray(b'\x00\x07\x0e\x15')
    keyboxin=bytearray([k1[0]^k2[0]^k3[0]^ck0[0], k1[1]^k2[1]^k3[1]^ck0[1], k1[2]^k2[2]^k3[2]^ck0[2], k1[3]^k2[3]^k3[3]^ck0[3]])
    keyboxout=bytearray([sbox[keyboxin[0]], sbox[keyboxin[1]], sbox[keyboxin[2]], sbox[keyboxin[3]]])
    left7=bytearray([keyboxout[2]<<7&255, keyboxout[3]<<7&255, keyboxout[0]<<7&255, keyboxout[1]<<7&255])
    right1=bytearray([keyboxout[3]>>1, keyboxout[0]>>1, keyboxout[1]>>1, keyboxout[2]>>1])
    left5=bytearray([keyboxout[1]<<5&255, keyboxout[2]<<5&255, keyboxout[3]<<5&255, keyboxout[0]<<5&255])
    right3=bytearray([keyboxout[2]>>3, keyboxout[3]>>3, keyboxout[0]>>3, keyboxout[1]>>3])
    rotation23=bytearray([left7[0]^right1[0], left7[1]^right1[1], left7[2]^right1[2], left7[3]^right1[3]])                   
    rotation13=bytearray([left5[0]^right3[0], left5[1]^right3[1], left5[2]^right3[2], left5[3]^right3[3]])
    linear=bytearray([keyboxout[0]^rotation13[0]^rotation23[0], keyboxout[1]^rotation13[1]^rotation23[1], keyboxout[2]^rotation13[2]^rotation23[2], keyboxout[3]^rotation13[3]^rotation23[3]])
    rk0=bytearray([k0[0]^linear[0], k0[1]^linear[1], k0[2]^linear[2], k0[3]^linear[3]])
    p=bytearray([p1[0]^p2[0]^p3[0], p1[1]^p2[1]^p3[1], p1[2]^p2[2]^p3[2], p1[3]^p2[3]^p3[3]])
    pboxin=bytearray([p1[0]^p2[0]^p3[0]^rk0[0], p1[1]^p2[1]^p3[1]^rk0[1], p1[2]^p2[2]^p3[2]^rk0[2], p1[3]^p2[3]^p3[3]^rk0[3]])
    label=bytearray([sbox[pboxin[0]], sbox[pboxin[1]], sbox[pboxin[2]], sbox[pboxin[3]]])
    hw = bytearray(bin(byte).count('1') for byte in label)
    return p, rk0, label, pboxin, k0, linear, p1, p2, p3, hw

In [9]:
def sbox_word_withmask(pt, key):
    #参数SBOX和FK
    mask=bytearray(b'\xa1\x3b\x6d\x4f\x92\xc8\x1f\x75\x83\xb6\xe2\x0a\x5c\xd9\x40\x3e\x7a\x19\xad\xb3\xc1\xef\x56\x29\x81\x37\x64\x5d\x98\xe7\xf2\x0b\xa9\x4c\x1e\x3f\xd5\x76\x85\xba\x92\x0f\x68\xd3\x4a\x7e\xbc\x51\xcd\x24\x3a\x67\x8f\xb1\xd0\xea\x1c\x49\x2d\x7f\x83\xa4\x5b\xe8\x34\x6b\x0e\xd7\xa2\x58\x9f\x21\xce\x7d\x40\x12\xe4\xa7\x5e\x19\x93\xbc\xd8\x0f\x62\x4e\xa5\x73\x1b\xf9\x88\xc4\x27\x3e\x60\x91\xda\x57\x1f\x86\x39\xb8\x4c\xef\x0a\x73\xc5\x9e\xd1\x28\x64\x7b\xaf\x43\x6d\x02\xe9\x1c\x57\xb4\x68\x2a\xde\x3f\x81\xc6\x5b\x90')
    sbox=bytearray(b'\xd6\x90\xe9\xfe\xcc\xe1\x3d\xb7\x16\xb6\x14\xc2\x28\xfb\x2c\x05\x2b\x67\x9a\x76\x2a\xbe\x04\xc3\xaa\x44\x13\x26\x49\x86\x06\x99\x9c\x42\x50\xf4\x91\xef\x98\x7a\x33\x54\x0b\x43\xed\xcf\xac\x62\xe4\xb3\x1c\xa9\xc9\x08\xe8\x95\x80\xdf\x94\xfa\x75\x8f\x3f\xa6\x47\x07\xa7\xfc\xf3\x73\x17\xba\x83\x59\x3c\x19\xe6\x85\x4f\xa8\x68\x6b\x81\xb2\x71\x64\xda\x8b\xf8\xeb\x0f\x4b\x70\x56\x9d\x35\x1e\x24\x0e\x5e\x63\x58\xd1\xa2\x25\x22\x7c\x3b\x01\x21\x78\x87\xd4\x00\x46\x57\x9f\xd3\x27\x52\x4c\x36\x02\xe7\xa0\xc4\xc8\x9e\xea\xbf\x8a\xd2\x40\xc7\x38\xb5\xa3\xf7\xf2\xce\xf9\x61\x15\xa1\xe0\xae\x5d\xa4\x9b\x34\x1a\x55\xad\x93\x32\x30\xf5\x8c\xb1\xe3\x1d\xf6\xe2\x2e\x82\x66\xca\x60\xc0\x29\x23\xab\x0d\x53\x4e\x6f\xd5\xdb\x37\x45\xde\xfd\x8e\x2f\x03\xff\x6a\x72\x6d\x6c\x5b\x51\x8d\x1b\xaf\x92\xbb\xdd\xbc\x7f\x11\xd9\x5c\x41\x1f\x10\x5a\xd8\x0a\xc1\x31\x88\xa5\xcd\x7b\xbd\x2d\x74\xd0\x12\xb8\xe5\xb4\xb0\x89\x69\x97\x4a\x0c\x96\x77\x7e\x65\xb9\xf1\x09\xc5\x6e\xc6\x84\x18\xf0\x7d\xec\x3a\xdc\x4d\x20\x79\xee\x5f\x3e\xd7\xcb\x39\x48')                   
    fk0=bytearray(b'\xa3\xb1\xba\xc6')
    fk1=bytearray(b'\x56\xaa\x33\x50')
    fk2=bytearray(b'\x67\x7d\x91\x97')
    fk3=bytearray(b'\xb2\x70\x22\xdc')
    #传入明文
    p1=bytearray([pt[4],pt[5],pt[6],pt[7]])
    p2=bytearray([pt[8],pt[9],pt[10],pt[11]])
    p3=bytearray([pt[12],pt[13],pt[14],pt[15]])
    #传入主密钥
    mk0=bytearray([key[0],key[1],key[2],key[3]])
    mk1=bytearray([key[4],key[5],key[6],key[7]])
    mk2=bytearray([key[8],key[9],key[10],key[11]])
    mk3=bytearray([key[12],key[13],key[14],key[15]])
    #主密钥和FK逐比特异或
    k0=bytearray([mk0[0]^fk0[0], mk0[1]^fk0[1], mk0[2]^fk0[2], mk0[3]^fk0[3]])
    k1=bytearray([mk1[0]^fk1[0], mk1[1]^fk1[1], mk1[2]^fk1[2], mk1[3]^fk1[3]])
    k2=bytearray([mk2[0]^fk2[0], mk2[1]^fk2[1], mk2[2]^fk2[2], mk2[3]^fk2[3]])
    k3=bytearray([mk3[0]^fk3[0], mk3[1]^fk3[1], mk3[2]^fk3[2], mk3[3]^fk3[3]])
    #参数CK
    ck0=bytearray(b'\x00\x07\x0e\x15')
    keyboxin=bytearray([k1[0]^k2[0]^k3[0]^ck0[0], k1[1]^k2[1]^k3[1]^ck0[1], k1[2]^k2[2]^k3[2]^ck0[2], k1[3]^k2[3]^k3[3]^ck0[3]])
    keyboxout=bytearray([sbox[keyboxin[0]], sbox[keyboxin[1]], sbox[keyboxin[2]], sbox[keyboxin[3]]])
    left7=bytearray([keyboxout[2]<<7&255, keyboxout[3]<<7&255, keyboxout[0]<<7&255, keyboxout[1]<<7&255])
    right1=bytearray([keyboxout[3]>>1, keyboxout[0]>>1, keyboxout[1]>>1, keyboxout[2]>>1])
    left5=bytearray([keyboxout[1]<<5&255, keyboxout[2]<<5&255, keyboxout[3]<<5&255, keyboxout[0]<<5&255])
    right3=bytearray([keyboxout[2]>>3, keyboxout[3]>>3, keyboxout[0]>>3, keyboxout[1]>>3])
    rotation23=bytearray([left7[0]^right1[0], left7[1]^right1[1], left7[2]^right1[2], left7[3]^right1[3]])                   
    rotation13=bytearray([left5[0]^right3[0], left5[1]^right3[1], left5[2]^right3[2], left5[3]^right3[3]])
    linear=bytearray([keyboxout[0]^rotation13[0]^rotation23[0], keyboxout[1]^rotation13[1]^rotation23[1], keyboxout[2]^rotation13[2]^rotation23[2], keyboxout[3]^rotation13[3]^rotation23[3]])
    rk0=bytearray([k0[0]^linear[0], k0[1]^linear[1], k0[2]^linear[2], k0[3]^linear[3]])
    p=bytearray([p1[0]^p2[0]^p3[0], p1[1]^p2[1]^p3[1], p1[2]^p2[2]^p3[2], p1[3]^p2[3]^p3[3]])
    pboxin=bytearray([p1[0]^p2[0]^p3[0]^rk0[0], p1[1]^p2[1]^p3[1]^rk0[1], p1[2]^p2[2]^p3[2]^rk0[2], p1[3]^p2[3]^p3[3]^rk0[3]])
    label=bytearray([sbox[pboxin[0]], sbox[pboxin[1]], sbox[pboxin[2]], sbox[pboxin[3]]])
    m1=bytearray([mask[0], mask[0], mask[0], mask[0]])
    m2=bytearray([sbox[m1[0]], sbox[m1[0]], sbox[m1[0]], sbox[m1[0]]])
    masked_input=bytearray([keyboxin[0]^m1[0], keyboxin[0]^m1[0], keyboxin[0]^m1[0], keyboxin[0]^m1[0]])
    share1=bytearray([sbox[masked_input[0]], sbox[masked_input[0]], sbox[masked_input[0]], sbox[masked_input[0]]])
    share2=m2
    return p, rk0, label, pboxin, k0, linear, p1, p2, p3, share1, share2


In [ ]:
f = h5py.File('sm4_raw_traces_1.hdf5', 'w')
rkgroup = f.create_group("rkgroup")
fkgroup = f.create_group("fkgroup")
traces_num_random=100000
traces_num_fixed=10000
scope.adc.samples = 23000
sample_num=scope.adc.samples
scope.adc.presamples=0
scope.gain.db = 0

rk_pt = rkgroup.create_dataset("ptinbox", (traces_num_random, 4), dtype='i8')
rk_key = rkgroup.create_dataset("roundkey", (traces_num_random, 4), dtype='i8')
rk_sbox = rkgroup.create_dataset("sbox", (traces_num_random, 4), dtype='i8')
rk_traces = rkgroup.create_dataset("traces", (traces_num_random, sample_num), dtype='f')

fk_pt = fkgroup.create_dataset("ptinbox", (traces_num_fixed, 4), dtype='i8')
fk_key = fkgroup.create_dataset("roundkey", (traces_num_fixed, 4), dtype='i8')
fk_sbox = fkgroup.create_dataset("sbox", (traces_num_fixed, 4), dtype='i8')
fk_traces = fkgroup.create_dataset("traces", (traces_num_fixed, sample_num), dtype='f')

def capture_trace(_ignored=None):
    ktp = cw.ktp.Basic()
    ktp.fixed_key=False
    key, text = ktp.next()
    return cw.capture_trace(scope, target, text, key)

for i in tqdm(range(traces_num_random)):
   
    trace = capture_trace()
    rk_pt[i, :], rk_key[i, :], rk_sbox[i, :]=sbox_word1(bytearray(trace.textin), bytearray(trace.key))
    rk_traces[i, :]=trace.wave

def capture_trace_fixed(_ignored=None):
    ktp = cw.ktp.Basic()
    ktp.fixed_key=True
    key, text = ktp.next()
    return cw.capture_trace(scope, target, text, key)
    
for i in tqdm(range(traces_num_fixed)):
   
    trace = capture_trace_fixed()
    fk_pt[i, :], fk_key[i, :], fk_sbox[i, :]=sbox_word1(bytearray(trace.textin), bytearray(trace.key))
    fk_traces[i, :]=trace.wave
    
    
f.close()

In [ ]:
f = h5py.File('masked_sm4_raw_traces_0627.hdf5', 'w')
rkgroup = f.create_group("rkgroup")
fkgroup = f.create_group("fkgroup")
traces_num_random=100000
traces_num_fixed=10000
scope.adc.samples = 23000
sample_num=scope.adc.samples
scope.adc.presamples=0
scope.gain.db = 0
######################################################################################################
rk_pt = rkgroup.create_dataset("ptinbox", (traces_num_random, 4), dtype='i8')
rk_key = rkgroup.create_dataset("roundkey", (traces_num_random, 4), dtype='i8')
rk_sbox = rkgroup.create_dataset("sbox", (traces_num_random, 4), dtype='i8')
rk_pboxin = rkgroup.create_dataset("pboxin", (traces_num_random, 4), dtype='i8')
rk_k0 = rkgroup.create_dataset("k0", (traces_num_random, 4), dtype='i8')
rk_linear = rkgroup.create_dataset("linear", (traces_num_random, 4), dtype='i8')
rk_p1 = rkgroup.create_dataset("p1", (traces_num_random, 4), dtype='i8')
rk_p2 = rkgroup.create_dataset("p2", (traces_num_random, 4), dtype='i8')
rk_p3 = rkgroup.create_dataset("p3", (traces_num_random, 4), dtype='i8')
rk_hw = rkgroup.create_dataset("hw", (traces_num_random, 4), dtype='i8')
rk_traces = rkgroup.create_dataset("traces", (traces_num_random, sample_num), dtype='f')
#####################################################################################################
fk_pt = fkgroup.create_dataset("ptinbox", (traces_num_fixed, 4), dtype='i8')
fk_key = fkgroup.create_dataset("roundkey", (traces_num_fixed, 4), dtype='i8')
fk_sbox = fkgroup.create_dataset("sbox", (traces_num_fixed, 4), dtype='i8')
fk_pboxin = fkgroup.create_dataset("pboxin", (traces_num_fixed, 4), dtype='i8')
fk_k0 = fkgroup.create_dataset("k0", (traces_num_fixed, 4), dtype='i8')
fk_linear = fkgroup.create_dataset("linear", (traces_num_fixed, 4), dtype='i8')
fk_p1 = fkgroup.create_dataset("p1", (traces_num_fixed, 4), dtype='i8')
fk_p2 = fkgroup.create_dataset("p2", (traces_num_fixed, 4), dtype='i8')
fk_p3 = fkgroup.create_dataset("p3", (traces_num_fixed, 4), dtype='i8')
fk_hw = fkgroup.create_dataset("hw", (traces_num_fixed, 4), dtype='i8')
fk_traces = fkgroup.create_dataset("traces", (traces_num_fixed, sample_num), dtype='f')


########################################################################################################
def capture_trace(_ignored=None):
    ktp = cw.ktp.Basic()
    ktp.fixed_key=False
    key, text = ktp.next()
    return cw.capture_trace(scope, target, text, key)
for i in tqdm(range(traces_num_random)):
    trace = capture_trace()
    rk_pt[i, :], rk_key[i, :], rk_sbox[i, :], rk_pboxin[i, :], rk_k0[i, :], rk_linear[i, :], rk_p1[i, :], rk_p2[i, :], rk_p3[i, :], rk_hw[i, :]=sbox_word2(bytearray(trace.textin), bytearray(trace.key))
    rk_traces[i, :]=trace.wave

def capture_trace_fixed(_ignored=None):
    ktp = cw.ktp.Basic()
    ktp.fixed_key=True
    key, text = ktp.next()
    return cw.capture_trace(scope, target, text, key)

for i in tqdm(range(traces_num_fixed)):
    trace = capture_trace_fixed()
    fk_pt[i, :], fk_key[i, :], fk_sbox[i, :], fk_pboxin[i, :], fk_k0[i, :], fk_linear[i, :], fk_p1[i, :], fk_p2[i, :], fk_p3[i, :], fk_hw[i, :]=sbox_word2(bytearray(trace.textin), bytearray(trace.key))
    fk_traces[i, :]=trace.wave    

    
    
f.close()

In [ ]:

f = h5py.File('sm4_mask_raw_traces_for_snr_3.hdf5', 'w')
rkgroup = f.create_group("rkgroup")
traces_num_random=100000
scope.adc.samples = 23000
sample_num=scope.adc.samples
scope.adc.presamples=0
scope.gain.db = 0

rk_pt = rkgroup.create_dataset("ptinbox", (traces_num_random, 4), dtype='i8')
rk_key = rkgroup.create_dataset("roundkey", (traces_num_random, 4), dtype='i8')
rk_sbox = rkgroup.create_dataset("sbox", (traces_num_random, 4), dtype='i8')
rk_pboxin = rkgroup.create_dataset("pboxin", (traces_num_random, 4), dtype='i8')
rk_k0 = rkgroup.create_dataset("k0", (traces_num_random, 4), dtype='i8')
rk_linear = rkgroup.create_dataset("linear", (traces_num_random, 4), dtype='i8')
rk_p1 = rkgroup.create_dataset("p1", (traces_num_random, 4), dtype='i8')
rk_p2 = rkgroup.create_dataset("p2", (traces_num_random, 4), dtype='i8')
rk_p3 = rkgroup.create_dataset("p3", (traces_num_random, 4), dtype='i8')
share1 = rkgroup.create_dataset("share1", (traces_num_random, 4), dtype='i8')
share2 = rkgroup.create_dataset("share2", (traces_num_random, 4), dtype='i8')
rk_traces = rkgroup.create_dataset("traces", (traces_num_random, sample_num), dtype='f')


def capture_trace(_ignored=None):
    ktp = cw.ktp.Basic()
    ktp.fixed_key=False
    key, text = ktp.next()
    return cw.capture_trace(scope, target, text, key)
for i in tqdm(range(traces_num_random)):
    trace = capture_trace()
    rk_pt[i, :], rk_key[i, :], rk_sbox[i, :], rk_pboxin[i, :], rk_k0[i, :], rk_linear[i, :], rk_p1[i, :], rk_p2[i, :], rk_p3[i, :], share1[i, :], share2[i, :]=sbox_word_withmask(bytearray(trace.textin), bytearray(trace.key))
    rk_traces[i, :]=trace.wave
    
    
f.close()

In [38]:
print(scope.clock)

adc_src       = clkgen_x1
adc_phase     = 0
adc_freq      = 8384273
adc_rate      = 8384273.0
adc_locked    = False
freq_ctr      = 0
freq_ctr_src  = extclk
clkgen_src    = system
extclk_freq   = 10000000
clkgen_mul    = 20
clkgen_div    = 229
clkgen_freq   = 8384279.475982533
clkgen_locked = True



In [22]:
scope.default_setup()

In [2]:
help(cw.capture_trace)

Help on function capture_trace in module chipwhisperer:

capture_trace(scope: Union[chipwhisperer.capture.scopes.OpenADC.OpenADC, chipwhisperer.capture.scopes.cwnano.CWNano], target: Union[chipwhisperer.capture.targets.CW305.CW305, chipwhisperer.capture.targets.CW305_ECC.CW305_ECC, chipwhisperer.capture.targets.CW310.CW310, chipwhisperer.capture.targets.SimpleSerial.SimpleSerial, chipwhisperer.capture.targets.SimpleSerial2.SimpleSerial2, chipwhisperer.capture.targets.SimpleSerial2.SimpleSerial2_CDC], plaintext: chipwhisperer.common.utils.util.bytearray, key: Optional[chipwhisperer.common.utils.util.bytearray] = None, ack: bool = True, poll_done: bool = False, as_int: bool = False, always_send_key=False) -> Optional[chipwhisperer.common.traces.Trace]
    Capture a trace, sending plaintext and key
    
    Does all individual steps needed to capture a trace (arming the scope
    sending the key/plaintext, getting the trace data back, etc.). Uses
    target.output_len as the length of the